In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test_bqCt9Pv.csv')
train.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [3]:
from datetime import date,datetime

def calculate_age(born):
    
    dd,mm,yy = born.split('-')
    if int(yy)>19:
        yy = ''.join(['19',yy])
    else:
        yy = ''.join(['20',yy])
    
    born = "-".join([dd,mm,yy])
    born = datetime.strptime(born, '%d-%m-%Y').date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [4]:
X = train.copy()
X_test = test.copy()

X['age'] = X['Date.of.Birth'].apply(lambda x: calculate_age(x))
X_test['age'] = X_test['Date.of.Birth'].apply(lambda x: calculate_age(x))

In [5]:
def get_months(val):
    yr,mon = val.split()
    yr = int(yr[:-3])*12
    mon = int(mon[:-3])
    tot = yr+mon
    return tot

In [6]:
X['AVERAGE.ACCT.AGE'] = X['AVERAGE.ACCT.AGE'].apply(lambda x: get_months(x))
X['CREDIT.HISTORY.LENGTH'] = X['CREDIT.HISTORY.LENGTH'].apply(lambda x: get_months(x))

X_test['AVERAGE.ACCT.AGE'] = X_test['AVERAGE.ACCT.AGE'].apply(lambda x: get_months(x))
X_test['CREDIT.HISTORY.LENGTH'] = X_test['CREDIT.HISTORY.LENGTH'].apply(lambda x: get_months(x))

In [7]:
labels = ['UniqueID','supplier_id','manufacturer_id','Current_pincode_ID',\
         'Date.of.Birth','Employment.Type','DisbursalDate','Employee_code_ID',\
         'MobileNo_Avl_Flag','Aadhar_flag','PAN_flag','VoterID_flag','Driving_flag',\
         'Passport_flag','PERFORM_CNS.SCORE.DESCRIPTION','branch_id']

X = X.drop(labels=labels,axis=1)
X_test = X_test.drop(labels=labels,axis=1)

X.head()

,disbursed_amount,asset_cost,ltv,State_ID,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,age
0,50578,58400,89.55,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35
1,47145,65550,73.23,6,598,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,23,23,0,1,33
2,53278,61360,89.63,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33
3,57513,66113,88.48,6,305,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,8,15,1,1,25
4,52378,60300,88.39,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,41


In [8]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [9]:
y=X['loan_default'].values

X = X.drop(labels=['loan_default'],axis=1)

continuous_cols = ['disbursed_amount', 'asset_cost', 'ltv',
       'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES','age']

categorical_cols = ['State_ID']

mapper = DataFrameMapper(  
    [([continuous_col], StandardScaler()) for continuous_col in continuous_cols] +
    [([categorical_col], OneHotEncoder()) for categorical_col in categorical_cols])

pipe = Pipeline([('mapper',mapper)])

pipe.fit(X)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by Stand

Pipeline(memory=None,
     steps=[('mapper', DataFrameMapper(default=False, df_out=False,
        features=[(['disbursed_amount'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['asset_cost'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['ltv'], StandardScaler(copy=True, with_mean=True, with_std=...andle_unknown='error',
       n_values=None, sparse=True))],
        input_df=False, sparse=False))])

In [10]:
X = pipe.transform(X)
X_test = pipe.transform(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by Stand

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.1,random_state=42)

In [12]:
# from sklearn.linear_model import LogisticRegression

In [13]:
# lrc = LogisticRegression(random_state = 42,n_jobs=-1,max_iter=1000)

In [14]:
# lrc.fit(X_train,y_train)

In [15]:
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score

In [16]:
# y_pred = lrc.predict_proba(X_val)

In [17]:
# y_pred = y_pred[:,0]

In [18]:
# def best_score(y_val,y_pred):
#     th = []
#     scores =[]
#     for thresh in np.arange(0.1, 0.601, 0.01):
#         thresh = np.round(thresh,2)
#         th.append(thresh)
#         score = f1_score(y_val,(y_pred>thresh).astype(int))
#         scores.append(score)
#     return np.max(scores),th[(np.argmax(scores))]
        
    

In [19]:
# score,threshold = best_score(y_val,y_pred)

# print(score)
# print(threshold)

In [20]:
# from keras import backend as K

# def f1(y_true, y_pred):
#     def recall(y_true, y_pred):
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + K.epsilon())
#         return recall

#     def precision(y_true, y_pred):
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision
    
#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [21]:
# X.shape

In [22]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

In [23]:
# model = Sequential()
# model.add(Dense(256,input_dim=128,activation='relu',))
# model.add((Dropout(0.4)))
# model.add(Dense(64,activation='relu'))
# model.add(Dense(32,activation='relu'))
# model.add(Dropout(rate=0.4))
# model.add(Dense(1,activation='sigmoid'))

# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', f1])
# model.summary()

In [24]:
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# checkpoints = ModelCheckpoint('model.h5',monitor='val_f1',mode='max',save_best_only='True',verbose=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_f1', factor=0.1, patience=2, verbose=1, min_lr=0.000001)

In [25]:
# batch_size = 128
# epochs = 10

In [26]:
# history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
#                     validation_data=[X_val, y_val], callbacks=[checkpoints, reduce_lr])

In [27]:
from sklearn.svm import LinearSVC

In [28]:
svc = LinearSVC(C=1.2,class_weight='balanced',random_state=42,max_iter=15000)

In [29]:
svc.fit(X_train,y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.2, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=15000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [30]:
pred = svc.predict(X_val)

In [31]:
print(f1_score(y_val,pred))
print(confusion_matrix(y_val,pred))

0.3899921054229672
[[10060  8169]
 [ 1876  3211]]


In [32]:
pred_test = svc.predict(X_test)

In [33]:
test1 = test.copy()

test1['loan_default'] = pred_test
test1 = test1.loc[:,['UniqueID','loan_default']]



In [34]:
test1.head()

,UniqueID,loan_default
0,655269,1
1,723482,1
2,758529,1
3,763449,1
4,708663,1


In [35]:
test1.to_csv('submission.csv',index=False)